<a href="https://colab.research.google.com/github/luci4-4/ML/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5%20%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas seaborn scikit-learn
!pip install pandas

# Домашнее задание 3

1. Выбрать номинальное значение как целевую переменную
2. Провести разведочный анализ **включая** анализ зависимости между номинальными переменными (chi2)
3. Выбрать `best_features`
4. Сделать нормализацию данных (scaler, onehotencoding)
5. Обучить модель логистической регрессии
6. Вывести метрики качества, описать полученный результат (также вывести коэфициенты модели)

In [30]:
import pandas as pd
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [10]:
df = pd.read_csv("/content/StudentPerformanceFactors.csv")

In [11]:
df

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6602,25,69,High,Medium,No,7,76,Medium,Yes,1,High,Medium,Public,Positive,2,No,High School,Near,Female,68
6603,23,76,High,Medium,No,8,81,Medium,Yes,3,Low,High,Public,Positive,2,No,High School,Near,Female,69
6604,20,90,Medium,Low,Yes,6,65,Low,Yes,3,Low,Medium,Public,Negative,2,No,Postgraduate,Near,Female,68
6605,10,86,High,High,Yes,6,91,High,Yes,2,Low,Medium,Private,Positive,3,No,High School,Far,Female,68


1. Была выбрана в качестве целевой переменной - Peer_Influence

2. Провести разведочный анализ включая анализ зависимости между номинальными переменными (chi2)

In [15]:
cat_columns = df.select_dtypes(include='object').columns.tolist()
print(cat_columns)

['Parental_Involvement', 'Access_to_Resources', 'Extracurricular_Activities', 'Motivation_Level', 'Internet_Access', 'Family_Income', 'Teacher_Quality', 'School_Type', 'Peer_Influence', 'Learning_Disabilities', 'Parental_Education_Level', 'Distance_from_Home', 'Gender']


In [17]:
cat_columns = [col for col in cat_columns if col != 'Peer_Influence']


In [18]:
print(df.describe(include='object'))


       Parental_Involvement Access_to_Resources Extracurricular_Activities  \
count                  6607                6607                       6607   
unique                    3                   3                          2   
top                  Medium              Medium                        Yes   
freq                   3362                3319                       3938   

       Motivation_Level Internet_Access Family_Income Teacher_Quality  \
count              6607            6607          6607            6529   
unique                3               2             3               3   
top              Medium             Yes           Low          Medium   
freq               3351            6108          2672            3925   

       School_Type Peer_Influence Learning_Disabilities  \
count         6607           6607                  6607   
unique           2              3                     2   
top         Public       Positive                    No   
freq   

In [19]:
for col in cat_columns:
    table = pd.crosstab(df['Peer_Influence'], df[col])
    chi2, p, dof, expected = chi2_contingency(table)
    print(f"Зависимость между Peer_Influence и {col} — p-value: {p:.4f}, chi2: {chi2:.2f}")


Зависимость между Peer_Influence и Parental_Involvement — p-value: 0.2587, chi2: 5.29
Зависимость между Peer_Influence и Access_to_Resources — p-value: 0.7824, chi2: 1.75
Зависимость между Peer_Influence и Extracurricular_Activities — p-value: 0.0244, chi2: 7.42
Зависимость между Peer_Influence и Motivation_Level — p-value: 0.3258, chi2: 4.64
Зависимость между Peer_Influence и Internet_Access — p-value: 0.6856, chi2: 0.76
Зависимость между Peer_Influence и Family_Income — p-value: 0.6129, chi2: 2.68
Зависимость между Peer_Influence и Teacher_Quality — p-value: 0.4803, chi2: 3.48
Зависимость между Peer_Influence и School_Type — p-value: 0.5533, chi2: 1.18
Зависимость между Peer_Influence и Learning_Disabilities — p-value: 0.5239, chi2: 1.29
Зависимость между Peer_Influence и Parental_Education_Level — p-value: 0.1138, chi2: 7.45
Зависимость между Peer_Influence и Distance_from_Home — p-value: 0.4752, chi2: 3.52
Зависимость между Peer_Influence и Gender — p-value: 0.3047, chi2: 2.38


1. Результаты χ²-теста:

Для большинства переменных значение p-value выше 0.05 — это значит, что статистически значимой связи между типом влияния сверстников (Peer_Influence) и такими переменными, как уровень вовлеченности родителей, доступ к ресурсам, мотивация, наличие интернета, доход семьи, качество учителя, тип школы, наличие трудностей с обучением, уровень образования родителей, удалённость от дома, гендер — не обнаружено.

Исключение: Extracurricular_Activities (внеклассная активность). Здесь p-value = 0.0244 (меньше 0.05), то есть присутствует статистически значимая связь между участием во внеклассных мероприятиях и влиянием сверстников. Это может говорить о том, что участие во внеклассной жизни действительно связано с особенностями воздействия окружения.

2. Описание категориальных переменных:

Большинство учеников учатся в государственных школах (School_Type: Public — 4598 из 6607).

Наиболее часто встречающиеся мотивация, уровень качества учителя, уровень вовлеченности родителей и доступ к ресурсам — Medium.

В основном учащиеся имеют доступ к интернету — 6108 из 6607.

Большинство без трудностей в обучении (Learning_Disabilities: No — 5912).

В основном родители имеют среднее образование (Parental_Education_Level: High School — 3223).

Большинство учеников живет недалеко от школы (Distance_from_Home: Near — 3884).

Преобладают мальчики (Gender: Male — 3814 из 6607).

Peer_Influence чаще положительное (Positive — 2638 из 6607).

Общий вывод:

Среди всех номинальных признаков только включённость во внеклассные мероприятия (Extracurricular_Activities) показала статистически значимую связь с восприятием влияния сверстников.

Остальные факторы, такие как тип школы, уровень вовлеченности родителей, мотивация, доход и др., не показали существенного влияния на переменную Peer_Influence по результатам χ²-теста.

Это может стать основой для дальнейшего анализа или построения моделей: переменная Extracurricular_Activities будет важным фактором, остальные — менее значимы для этой цели.

3. Выбрать best_features

In [23]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder

# Кодируем целевую переменную числами
y = LabelEncoder().fit_transform(df['Peer_Influence'])

# Кодируем номинальные признаки в числовые (dummy/one-hot encoding)
X_cat = pd.get_dummies(df[cat_columns])

# Выборка лучших признаков (k=5 или любое другое число)
selector = SelectKBest(score_func=chi2, k='all')
selector.fit(X_cat, y)

scores = selector.scores_
feature_scores = pd.Series(scores, index=X_cat.columns)
best_features = feature_scores.sort_values(ascending=False)
print(best_features.head(10))


Parental_Education_Level_College    5.225968
Extracurricular_Activities_No       4.424369
Extracurricular_Activities_Yes      2.998639
Parental_Involvement_Low            2.916551
Motivation_Level_Medium             2.282349
Distance_from_Home_Moderate         2.240905
Teacher_Quality_Low                 2.009968
Parental_Involvement_High           1.418908
Family_Income_High                  1.396600
Gender_Female                       1.371915
dtype: float64


На основе результатов отбора признаков (feature selection) с помощью χ²-статистики, наиболее информативными признаками для прогнозирования значения переменной Peer_Influence являются:

Parental_Education_Level_College — уровень образования родителей (колледж)

Extracurricular_Activities_No — отсутствие участия во внеклассных активностях

Extracurricular_Activities_Yes — участие во внеклассных активностях

Parental_Involvement_Low — низкая вовлечённость родителей в обучение

Motivation_Level_Medium — средний уровень мотивации ученика

Эти признаки получили наибольшие значения статистики χ², что указывает на их значимый вклад в объяснение вариаций таргетной переменной Peer_Influence.
В практической задаче рекомендуется использовать именно их (и, возможно, ещё 1–2 следующих по значимости), чтобы построить максимально эффективную модель для предсказания влияния сверстников на основе ваших данных.

In [24]:
# Создаём dummy-признаки (one-hot encoding)
df_dummies = pd.get_dummies(df, columns=[
    'Parental_Education_Level',
    'Extracurricular_Activities',
    'Parental_Involvement',
    'Motivation_Level',
    'Distance_from_Home',
    'Teacher_Quality',
    'Family_Income',
    'Gender'
])

# Теперь список best_features подходит к полученному DataFrame
best_features = [
    'Parental_Education_Level_College',
    'Extracurricular_Activities_No',
    'Extracurricular_Activities_Yes',
    'Parental_Involvement_Low',
    'Motivation_Level_Medium'
]

X_best = df_dummies[best_features]

4. Сделать нормализацию данных (scaler, onehotencoding)

In [27]:
# Формируем обучающую выборку из best_features
X_best = df_dummies[best_features]

# Если среди признаков есть числовые — их нужно нормализовать:
# Пример добавления числового признака:
# df_dummies['Some_Numeric_Column']

scaler = StandardScaler()

# Если у вас только one-hot признаки из категориальных переменных, scaler не нужен!

print(X_best.head())

   Parental_Education_Level_College  Extracurricular_Activities_No  \
0                             False                           True   
1                              True                           True   
2                             False                          False   
3                             False                          False   
4                              True                          False   

   Extracurricular_Activities_Yes  Parental_Involvement_Low  \
0                           False                      True   
1                           False                      True   
2                            True                     False   
3                            True                      True   
4                            True                     False   

   Motivation_Level_Medium  
0                    False  
1                    False  
2                     True  
3                     True  
4                     True  


 провёл нормализацию данных: преобразовал все выбранные категориальные переменные в one-hot формат с помощью pd.get_dummies. Теперь для каждого уникального значения переменной создан отдельный столбец, где указывается, присутствует ли это значение у данного наблюдения (True/False).

Мои признаки для обучения представляют бинарные столбцы, и дополнительная числовая нормализация для них не требуется — их можно напрямую использовать в модели машинного обучения. Благодаря этому данные пригодны для дальнейшего обучения и анализа, и я могу переходить к построению и тестированию модели на этом наборе признаков.

5. Обучить модель логистической регрессии

In [31]:
y = LabelEncoder().fit_transform(df['Peer_Influence'])

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_best, y, test_size=0.2, random_state=42)

# Обучаем модель логистической регрессии
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Получаем предсказания
y_pred = model.predict(X_test)

# Оцениваем результат
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.39107413010590014
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       271
           1       0.46      0.22      0.29       565
           2       0.37      0.81      0.51       486

    accuracy                           0.39      1322
   macro avg       0.28      0.34      0.27      1322
weighted avg       0.33      0.39      0.31      1322



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Общая точность (Accuracy) составила 0.39 — это довольно низкое значение, то есть модель предсказывает целевую переменную Peer_Influence правильно в 39% случаев.

Качество классификации по классам:

Для класса 0 (скорее всего один из вариантов Peer_Influence) модель не смогла ни разу предсказать этот класс правильно (precision, recall, f1-score — все равны 0). Значит, либо признаки почти не несут информации для данного класса, либо он недостаточно представлен в обучающей выборке.

Для класса 1 модель показывает невысокие показатели (precision — 0.46, recall — 0.22, f1 — 0.29), то есть часто ошибается и по этому классу.

Для класса 2 модель определяет его лучше: высокая полнота (recall — 0.81), но точность (precision — 0.37) и f1-score (0.51) средние — то есть модель часто маркирует этот класс, но не всегда правильно.

Средние и взвешенные метрики (macro avg, weighted avg) также низкие — меньше 0.4 по всем показателям.

Общий вывод:
Модель логистической регрессии на выбранных признаках характеризуется низким качеством. Это говорит о слабой предсказательной мощности выделенных признаков, наличии мультиклассового дисбаланса либо сложности самой задачи. Требуется либо подбор других (дополнительных) признаков, либо применение более сложной или иной модели, либо балансировка классов.

6. Вывести метрики качества, описать полученный результат (также вывести коэфициенты модели)

In [32]:
# Для многоклассовой задачи coef_ имеет размер n_classes x n_features
for i, class_coef in enumerate(model.coef_):
    print(f'Класс {i}:')
    for feat, coef in zip(X_best.columns, class_coef):
        print(f'{feat}: {coef:.3f}')
    print('-' * 25)


Класс 0:
Parental_Education_Level_College: 0.159
Extracurricular_Activities_No: -0.167
Extracurricular_Activities_Yes: -0.141
Parental_Involvement_Low: 0.096
Motivation_Level_Medium: -0.060
-------------------------
Класс 1:
Parental_Education_Level_College: -0.110
Extracurricular_Activities_No: 0.097
Extracurricular_Activities_Yes: 0.027
Parental_Involvement_Low: -0.045
Motivation_Level_Medium: 0.087
-------------------------
Класс 2:
Parental_Education_Level_College: -0.049
Extracurricular_Activities_No: 0.071
Extracurricular_Activities_Yes: 0.114
Parental_Involvement_Low: -0.051
Motivation_Level_Medium: -0.027
-------------------------


Вот мои выводы по значениям коэффициентов модели логистической регрессии:

Класс 0:
Наибольший положительный вклад даёт признак Parental_Education_Level_College (0.159) — наличие образования “колледж” у родителей немного увеличивает вероятность попадания наблюдения в этот класс.
Признаки Extracurricular_Activities_No и Extracurricular_Activities_Yes (оба отрицательные) — отсутствие или наличие внеклассных активностей снижает вероятность класса 0.
Parental_Involvement_Low (0.096) — низкая вовлечённость родителей также немного повышает вероятность этого класса.
Motivation_Level_Medium (–0.060) — средний уровень мотивации снижает вероятность этого класса.

Класс 1:
Наоборот, у признака Parental_Education_Level_College коэффициент отрицательный (–0.110), то есть образование колледж у родителей уменьшает вероятность класса 1.
Extracurricular_Activities_No (0.097) — отсутствие активности слегка увеличивает вероятность класса 1.
Motivation_Level_Medium (0.087) — положительный вклад среднего уровня мотивации в вероятность класса 1.

Класс 2:
Наиболее значимым положительным коэффициентом обладает Extracurricular_Activities_Yes (0.114) — участие во внеклассной деятельности увеличивает вероятность класса 2.
Extracurricular_Activities_No (0.071) тоже положительный, но слабее.
Остальные признаки имеют маленький по модулю вклад.

Общий итог:

Влияние отдельных признаков на все классы невелико (коэффициенты близки к нулю), что говорит о слабой линейной связи между признаками и классами.

Самый значимый (по модулю) признак — это Parental_Education_Level_College для класса 0 (0.159, наибольшая по модулю величина).

Для класса 2 (в котором recall самой модели наилучший), немалое влияние оказывает участие во внеклассных активностях.

Направление и величина коэффициента показывают качественную связь — положительное значение увеличивает вероятность соответствующего класса, отрицательное — уменьшает.

Вывод:
Признаки оказывают незначительное и разнонаправленное влияние на вероятность того, к какому классу относится Peer_Influence. Модель выявила связь между уровнем образования родителей, участием во внеклассных активностях и типом влияния сверстников, однако сила этих связей невелика, что отражается в невысоких метриках качества.
Для повышения прогностической мощности можно добавить новые признаки, провести feature engineering или использовать более сложные модели.